#  Den multiple linære regression model (MLR)

### Econometrics A (ØkA)

Wooldridge (Ch. 3)

Bertel Schjerning

Department of Economics, University of Copenhagen


# Part 1: Timeløn, uddannelse og erfaring (OLS estimation)

### Lønregression: Timeløn, uddannelse og erfaring
Estimation af lineær model for timeløn, uddannelse og erfaring:
$$
		\log(\text{wage}_i) = \beta _{0}+\beta _{1}\text{educ}_i+\beta_{2}\text{experience}_i+\beta _{3}\text{experience}^2_i+u_i
$$
- Vi benytter samme data fra Danmarks Statistik
    - Data indeholder $N=1.078$ tilfældigt udvalgte personer i 1994.
    - For hvert individ, $i$, har vi information om timelønnen, køn, antal årsuddannelse, single, alder og erhverserfaring. 
    - $\log(\text{wage}_i)$: Logartimen til timeløn i DKK i 1994 
    - $\text{educ}_i$: antal års uddannelse
    - $\text{experience}_i$: erhverserfaring målt i år
 
- Datasættet er afgrænset således:
    - 20-68 årige.			
	- Lønmodtagere.			
	- Timeløn på mere end 40 kr.

### Enable autoreload

In [1]:
# Sørger for at alle importerede python filer geninlæses ved import statements
# Nødvedigt, hvis ændinger skal tage effekt uden at genstarte Python Kernel
%load_ext autoreload
%autoreload 2

### Indlæs data, estimer model og print resulater 

In [2]:
import pandas as pd
import numpy as np
import mymlr as mlr # see mymlr.py

# Load the data and create variables
df = pd.read_stata(r"../data/wage.dta")   # Load data
df['const'] = 1                           # Add constant term
df['lwage'] = np.log(df['wage'])          # Log of wage
df['experience2'] = df['experience'] ** 2 # Add experience²

# Estimate the model using the mlr.ols function
mlr1 = mlr.ols(df[['const', 'educ']], df['lwage'])
mlr2 = mlr.ols(df[['const', 'educ', 'experience']], df['lwage'])
mlr3 = mlr.ols(df[['const', 'educ', 'experience', 'experience2']], df['lwage'])

# Print the summary using the mlr.summary() function
mlr.summary([mlr1,mlr2,mlr3])

                    Model 1  Model 2  Model 3
Dependent variable    lwage    lwage    lwage
             const   4.5604   4.3791   4.3155
                   (0.0337) (0.0350) (0.0396)
              educ   0.0282   0.0281   0.0274
                   (0.0028) (0.0027) (0.0027)
        experience            0.0125   0.0254
                            (0.0010) (0.0040)
       experience2                    -0.0004
                                     (0.0001)
         R_squared   0.0845   0.1948   0.2033
               TSS 111.2507 111.2507 111.2507
               RSS 101.8496  89.5751  88.6338
               ESS   9.4011  21.6756  22.6168
                 n     1078     1078     1078


# Part 2: OLS estimator og rangbetingelsen

### OLS estimator i matrix form

In [3]:
import pandas as pd
from numpy import linalg as la

def OLS(X, y):
    # Convert pandas DataFrame/Series to 2d-numpy arrays
    xvar = X.columns.tolist()
    X = X.values  # (n x k)
    y = y.values.reshape(-1, 1)  # (n x 1)
    
    # OLS estimates: β = (X'X)^(-1) X'y
    beta_hat = la.inv(X.T @ X) @ X.T @ y  # (k x 1)
    
    # Create a pandas DataFrame for clean output
    results_df = pd.DataFrame(data=beta_hat, index=xvar, columns=['Coefficients'])

    # Print the results using pandas
    print(results_df)
    
    return beta_hat

# Estimate the model
xvar = ['const', 'educ', 'experience', 'experience2']
beta_hat = OLS(df[xvar], df['lwage'])



             Coefficients
const            4.315456
educ             0.027435
experience       0.025363
experience2     -0.000408


### Rutine til til at kontrollere rangbetingelse og om nødevendigt fjerne lineært afhængie variable

In [4]:
from numpy import linalg as la
import pandas as pd

def remove_dependent_columns(X, tol=1e-10):
    """
    Remove linearly dependent columns from a pandas DataFrame X.
    
    Parameters:
    X (pd.DataFrame): The design matrix (n x k)
    tol (float): Tolerance for detecting linear dependence

    Returns:
    X_new (pd.DataFrame): Matrix with linearly dependent columns removed
    removed_columns (list): List of removed column names
    """
    
    # Check the rank of the matrix
    rank_X = la.matrix_rank(X.values)
    if rank_X == X.shape[1]:
        print("Matrix has full rank, no columns need to be removed.")
        return X, []
    
    print(f"Rank of X before removal: {rank_X} / {X.shape[1]}")

    # Perform QR decomposition to identify linearly dependent columns
    Q, R = la.qr(X.values)
    independent_columns = np.abs(np.diag(R)) > tol
    removed_columns = X.columns[~independent_columns]

    # Return matrix with only independent columns and list of removed columns
    X_new = X.iloc[:, independent_columns]
    
    print(f"Removed columns: {list(removed_columns)}")
    print(f"Rank of X after removal: {la.matrix_rank(X_new.values)} / {X_new.shape[1]}")

    return X_new, list(removed_columns)

### Rangbetingelsen og multikolinearitet

In [5]:
# Create a new variable 'exper_educ' as the sum of education and experience
df['exper_educ'] = df['experience'] + df['educ']  # Sum af udd og erfaring

# Define the full set of variables, including potential linear dependencies
xvar = ['const', 'educ', 'experience', 'experience2', 'exper_educ']

# Check the rank of the matrix and remove dependent columns if necessary
X_reduced, removed_cols = remove_dependent_columns(df[xvar])

# OLS after removing linear dependent variables
print('OLS - after removing linear dependent variables')
beta_hat_reduced = OLS(X_reduced, df['lwage'])

# OLS without removing linear dependent variables (on full matrix)
print('\nOLS - without removing linear dependent variables')
beta_hat_full = OLS(df[xvar], df['lwage'])

Rank of X before removal: 4 / 5
Removed columns: ['exper_educ']
Rank of X after removal: 4 / 4
OLS - after removing linear dependent variables
             Coefficients
const            4.315456
educ             0.027435
experience       0.025363
experience2     -0.000408

OLS - without removing linear dependent variables
             Coefficients
const           27.180540
educ            -1.755717
experience      -0.143985
experience2     -0.000408
exper_educ       0.084471


## Hvad hvis X ikke har fuld rang?

### 1. Multikollinearitet og reduceret rang:
- **Multikollinearitet** opstår, når forklarende variable er stærkt korrelerede. 
- I modellen er `exper_educ` (sum af `educ` og `experience`) perfekt lineært afhængig af de andre variable, hvilket fører til **reduceret rang**, hvor \( X'X \) ikke kan inverteres.

### 2. Fjernelse af afhængige variable:
- Ved at fjerne `exper_educ` får vi en model med fuld rang, og OLS-estimationen fungerer korrekt.
- Lineært afhængige variable identificeres ved brug af **QR-dekomponering**, som gør det muligt at fjerne kolonner med små diagonalelementer i \( R \)-matricen.


### 3. Effekten af multikollinearitet:
- Inkludering af `exper_educ` skaber store udsving i OLS-koefficienterne, såsom en høj konstant og en negativ koefficient for `educ`.
- Dette skyldes modellens ustabilitet ved multikollinearitet.

### 4. Læringspunkter:
- **Rangtjek** er vigtigt i regressionsmodeller for at sikre pålidelige estimater.
- Fjernelse af lineært afhængige variable giver en stabil model.


# Part 3: Timeløn, uddannelse og erfaring (Frisch-Waugh)

# Frisch-Waugh-Lovell Teoremet: Implementering

## Modeloversigt:

Vi undersøger en **multipel lineær regressionsmodel** til at forudsige log-løn (`lwage`) baseret på uddannelse (`educ`), erfaring (`experience`), og erfaring i anden potens (`experience2`).

Modellen er:
$$
y = \beta_0 + \beta_1 \text{uddannelse} + \beta_2 \text{erfaring} + \beta_3 \text{erfaring}^2 + u
$$

## Anvendelse af FWL Teoremet:

### Trin:
1. **Fuld model**: Regressér `lwage` på `educ`, `experience`, og `experience2`.
2. **Delvis regression**: Regressér `educ` på `experience` og `experience2`.
3. **Anden fase**: Regressér `lwage` på residualerne (`educ_r`) fra trin 2.

## Forventede Resultater:

- Koefficienten for `educ` i anden fase skal være den samme som i den fulde model.
- FWL-teoremet viser, at hvis man fjerner effekten af `experience` og `experience2`, vil estimatet for `educ` være uændret og isolere dens effekt på løn.


### Frisch-Waugh-Lovell (FWL) theorem using OLS regression

In [6]:
# Step 1: Estimate the full model
mlr_full = mlr.ols(df[['const', 'educ', 'experience', 'experience2']], df['lwage'])

# Step 2: Regress 'educ' on 'experience' and 'experience2' (partial regression)
educ_regression = mlr.ols(df[['const', 'experience', 'experience2']], df['educ'])

# Use the residuals of 'educ' from the partial regression (this is done automatically by mlr.ols)
df['educ_r'] = educ_regression['residuals'].flatten()  # Residuals are already computed by mlr.ols

# Step 3: Regress 'lwage' on 'educ_r' (second-stage regression)
mlr_step2 = mlr.ols(df[['const', 'educ_r']], df['lwage'])

# Step 4: Print the summary for all models
mlr.summary([mlr_full, educ_regression, mlr_step2], options=['beta_hat', 'R_squared'])


                    Model 1  Model 2  Model 3
Dependent variable    lwage     educ    lwage
             const   4.3155  10.8176   4.8834
                   (0.0396) (0.3142) (0.0094)
        experience   0.0254   0.1123         
                   (0.0040) (0.0454)         
       experience2  -0.0004  -0.0035         
                   (0.0001) (0.0014)         
              educ   0.0274                  
                   (0.0027)                  
            educ_r                     0.0274
                                     (0.0028)
         R_squared   0.2033   0.0060   0.0794
